In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor


pd.set_option('display.max_columns', None)


In [75]:
!ls *.csv

exercice_data.csv


In [79]:
df.columns

Index(['StudentID', 'FirstName', 'FamilyName', 'sex', 'age', 'address',
       'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason',
       'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup',
       'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet',
       'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health',
       'absences', 'FinalGrade'],
      dtype='object')

In [76]:
df = pd.read_csv("exercice_data.csv", sep=None, encoding='latin1')
df.head()

/tmp/ipykernel_7177/4032176105.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv("exercice_data.csv", sep=None, encoding='latin1')


,StudentID,FirstName,FamilyName,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,FinalGrade
0,0,Rafael,Morais,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,6
1,1,Lara,Braga,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,6
2,2,Gonçalo,Azevedo,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,10
3,3,Lara,Perna,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15
4,4,João,Pires,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,10


In [77]:
df.shape

(357, 33)

In [78]:
df.isna().sum()

StudentID     0
FirstName     0
FamilyName    0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
FinalGrade    0
dtype: int64

In [29]:
df.Medu.value_counts()

Medu
4    125
3     90
2     89
1     50
0      3
Name: count, dtype: int64

In [26]:
df.address.value_counts()

address
U    279
R     78
Name: count, dtype: int64

## Composite score 

In [11]:
# Define weights for each indicator
weights = {
    'absences': 0.1,
    'studytime': 0.1,
    'Dalc': 0.1,
    'Walc': 0.1,
    'health': 0.1,
    'failures': 0.1,
    'internet': 0.1,
    'famsup': 0.1,
    'schoolsup': 0.1
}

# Select only numerical columns for normalization
numerical_data = df.select_dtypes(include=['number'])

# Normalize each indicator to a scale of 0 to 1
max_values = numerical_data.max()
min_values = numerical_data.min()
normalized_data = (numerical_data - min_values) / (max_values - min_values)

# Compute the composite score using weighted sum
composite_score = (normalized_data * pd.Series(weights)).sum(axis=1)

# Add the composite score as a new column to your DataFrame
df['composite_score'] = composite_score

# Model 

In [80]:
# Define a dictionary to map "yes" to 1 and "no" to 0
replace_dict = {"yes": 1, "no": 0}

# Replace "yes" and "no" with 1 and 0 in the entire DataFrame
df.replace(replace_dict, inplace=True)

/tmp/ipykernel_7177/1804370698.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(replace_dict, inplace=True)


In [81]:
df.drop(columns=columns_to_encode)

,StudentID,FirstName,FamilyName,age,Medu,Fedu,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,FinalGrade
0,0,Rafael,Morais,18,4,4,2,2,0,1,0,0,0,1,1,0,0,4,3,4,1,1,3,6,6
1,1,Lara,Braga,17,1,1,1,2,0,0,1,0,0,0,1,1,0,5,3,3,1,1,3,4,6
2,2,Gonçalo,Azevedo,15,1,1,1,2,3,1,0,1,0,1,1,1,0,4,3,2,2,3,3,10,10
3,3,Lara,Perna,15,4,2,1,3,0,0,1,1,1,1,1,1,1,3,2,2,1,1,5,2,15
4,4,João,Pires,16,3,3,1,2,0,0,1,1,0,1,1,0,0,4,3,2,1,2,5,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,390,Alice,Barcellos,20,2,2,1,2,2,0,1,1,0,1,1,0,0,5,5,4,4,5,4,11,9
353,391,Rodrigo,Quintal,17,3,1,2,1,0,0,0,0,0,0,1,1,0,2,4,5,3,4,2,3,16
354,392,Carolina,Hiza,21,1,1,1,1,3,0,0,0,0,0,1,0,0,5,5,3,3,3,3,3,7
355,393,Benedita,Amaral,18,3,2,3,1,0,0,0,0,0,0,1,1,0,4,4,1,3,4,5,0,10


In [82]:
# Define columns to apply one-hot encoding
columns_to_encode = ['sex', 'address', 'famsize', 'Pstatus', 'reason', 'guardian', 'Mjob', 'Fjob']

# Define transformer for one-hot encoding
encoder = OneHotEncoder(drop='first')

# Define preprocessor for the pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', encoder, columns_to_encode)
    ],
    remainder='passthrough'
)

# Define pipeline with preprocessor and model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=5, random_state=42))
    #('model', Ridge(random_state=42))
    # ('model', XGBRegressor(random_state=42))

])

# Split data into features and target variable
X = df.drop(columns=['FinalGrade', 'StudentID', 'FirstName', 'FamilyName'])
y = df['FinalGrade'].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit the model on the training set
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate evaluation metrics
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Print evaluation metrics
print("Test set RMSE:", rmse)
print("Test set MAE:", mae)
print("Test set R-squared (R2):", r2)
print("Test set MSE:", mse)

Test set RMSE: 2.896459222173433
Test set MAE: 2.3218271613882906
Test set R-squared (R2): 0.170271601852508
Test set MSE: 8.38947602571353


/home/adelchiasta/venv/venv_ekinox/lib64/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [83]:
# # Define columns to apply one-hot encoding
# columns_to_encode = ['sex', 'address', 'famsize', 'Pstatus', 'reason', 'guardian', 'Mjob', 'Fjob']

# # Define transformer for one-hot encoding
# encoder = OneHotEncoder(drop='first')

# # Define preprocessor for the pipeline
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('onehot', encoder, columns_to_encode)
#     ],
#     remainder='passthrough'
# )

# # Define pipeline with preprocessor and model (using RandomForestRegressor)
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('reg', RandomForestRegressor(random_state=42))
# ])

# # Grid search parameters
# grid = [
#     {
#         "reg__n_estimators": [50, 100, 200],  # RandomForest hyperparameters
#         "reg__max_depth": [None, 10, 20],
#         "reg__min_samples_split": [2, 5, 10],
#         "reg__min_samples_leaf": [1, 2, 4],
#     },
# ]

# # Split data into features and target variable
# X = df.drop(columns=['FinalGrade', 'StudentID', 'FirstName', 'FamilyName'])
# y = df['FinalGrade'].values

# # Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Perform grid search without cross-validation
# grid_search = GridSearchCV(pipeline, grid, cv=1, scoring='neg_mean_squared_error', verbose=1)
# grid_search.fit(X_train, y_train)

# # Best parameters found during grid search
# print("Best parameters found:", grid_search.best_params_)

# # Make predictions on the test set using the best model
# y_pred = grid_search.predict(X_test)

# # Calculate evaluation metrics
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# mae = mean_absolute_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)

# # Print evaluation metrics
# print("Test set RMSE:", rmse)
# print("Test set MAE:", mae)
# print("Test set R-squared (R2):", r2)
# print("Test set MSE:", mse)


In [84]:
# Calculate evaluation metrics
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Print evaluation metrics
print("Test set RMSE:", rmse)
print("Test set MAE:", mae)
print("Test set R-squared (R2):", r2)
print("Test set MSE:", mse)

Test set RMSE: 2.896459222173433
Test set MAE: 2.3218271613882906
Test set R-squared (R2): 0.170271601852508
Test set MSE: 8.38947602571353


/home/adelchiasta/venv/venv_ekinox/lib64/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [85]:
# Make predictions on both train and test sets
y_train_pred = pipeline.predict(X_train)
y_test_pred = pipeline.predict(X_test)

# Calculate evaluation metrics for train set
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)

# Calculate evaluation metrics for test set
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Create DataFrame with metrics
metrics_df = pd.DataFrame({
    'Set': ['Train', 'Test'],
    'RMSE': [train_rmse, test_rmse],
    'MAE': [train_mae, test_mae],
    'R2': [train_r2, test_r2],
    'MSE': [train_mse, test_mse]
})

/home/adelchiasta/venv/venv_ekinox/lib64/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/adelchiasta/venv/venv_ekinox/lib64/python3.11/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [86]:
metrics_df

,Set,RMSE,MAE,R2,MSE
0,Train,1.414912,1.138363,0.809291,2.001976
1,Test,2.896459,2.321827,0.170272,8.389476
